Notebook for Tweet Cleaning and Initial Exploration

In [ ]:
!pip install wordcloud
!pip install plotly

In [ ]:
# Import packages **need to filter through and only import what's needed for this part
import re
import string
import collections
import ipywidgets
import cufflinks
import nltk.tokenize

import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

In [ ]:
pd.set_option('display.max_colwidth', None)

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

Cleaning Dataset

In [ ]:
# Import raw dataset
tweets_df = pd.read_csv('dummy data.csv')
tweets_df.head()

In [ ]:
def processed(text):
    # Remove links
    text = re.sub('http://\S+|https://\S+', '', text)
    text = re.sub('http[s]?://\S+', '', text)
    text = re.sub(r"http\S+", "", text)

    # Convert HTML references
    text = re.sub('&amp', 'and', text)
    text = re.sub('&lt', '<', text)
    text = re.sub('&gt', '>', text)
    text = re.sub('\xao', ' ', text)

    # Remove new line characters
    text = re.sub('[\r\n]+', ' ', text)
    
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)

    # Remove multiple space characters
    text = re.sub('\s+',' ', text)
    
    # Convert to lowercase
    text = text.lower()
    return text

In [ ]:
tweets_df['processed_text'] = tweets_df['text'].apply(processed)
tweets_df.head()

Hashtags

In [ ]:
# Create list of all hashtags in dataset
hashtags_list = []

for tweet in tweets_df['text']:
    # Extract hashtags from tweet content
    hashtags = re.findall(r"#\w+", tweet[2])
    # Append hashtags to list
    hashtags_list.extend(hashtags)

In [ ]:
# Count and visualize the most common hashtags present in the dataset

Mentions

Unigrams (incl. worldcloud w/wo keywords)

Bi-grams

Tri-grams